# Verifying claims I make in the writeup

## Setup

In [ ]:
import torch as t
# import torchtext
from tqdm import tqdm
import transformers

In [ ]:
DEVICE = "cuda" if t.cuda.is_available() else "cpu"
DEVICE

In [ ]:
gpt = transformers.GPT2LMHeadModel.from_pretrained("gpt2").to(DEVICE)

In [ ]:
tokenizer = transformers.GPT2TokenizerFast.from_pretrained("gpt2")

## How bad would it be to fix the mean and std that LayerNorm uses?

In [ ]:
(dataset,) = torchtext.datasets.EnWik9()

In [ ]:
class SaveForward:
    def __init__(self, layer):
        self.layer = layer

    def __enter__(self):
        self.remove_handle = self.layer.register_forward_hook(self.hook)
        return self

    def __exit__(self, exn_ty, exn_val, exn_tb):
        self.remove_handle.remove()

    def hook(self, module, module_input, module_output):
        self.saved_input = module_input
        self.saved_output = module_output   

In [ ]:
saved_stats = []
for _, data in zip(tqdm(range(1000)), dataset):
    input_ids = tokenizer.encode(data, return_tensors="pt").to(DEVICE)
    with t.no_grad():
        with SaveForward(gpt.transformer.ln_f) as saved:
            gpt(input_ids)
            (inp,) = saved.saved_input
            saved_stats.append(
                (t.mean(inp).item(), t.std(inp).item())
            )

In [ ]:
saved_stats[:20]

Seems like it might be pretty bad.

In [ ]:
gpt.transformer.ln_f